# Authentication

In [5]:
#!/usr/bin/python
## 6/16/2017 - remove PyQuery dependency
## 5/19/2016 - update to allow for authentication based on api-key, rather than username/pw
## See https://documentation.uts.nlm.nih.gov/rest/authentication.html for full explanation

import requests
#from pyquery import PyQuery as pq
import lxml.html as lh
from lxml.html import fromstring

uri="https://utslogin.nlm.nih.gov"
#option 1 - username/pw authentication at /cas/v1/tickets
auth_endpoint = "/cas/v1/tickets/"
#option 2 - api key authentication at /cas/v1/api-key
#auth_endpoint = "/cas/v1/api-key"

class Authentication:

   #def __init__(self, username,password):
   def __init__(self, username, password):
    self.username=username
    self.password=password
    #self.apikey=apikey
    self.service="http://umlsks.nlm.nih.gov"

   def gettgt(self):
     params = {'username': self.username,'password': self.password}
     #params = {'apikey': self.apikey}
     h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
     r = requests.post(uri+auth_endpoint,data=params,headers=h)
     response = fromstring(r.text)
     ## extract the entire URL needed from the HTML form (action attribute) returned - looks similar to https://utslogin.nlm.nih.gov/cas/v1/tickets/TGT-36471-aYqNLN2rFIJPXKzxwdTNC5ZT7z3B3cTAKfSc5ndHQcUxeaDOLN-cas
     ## we make a POST call to this URL in the getst method
     tgt = response.xpath('//form/@action')[0]
     return tgt

   def getst(self,tgt):

     params = {'service': self.service}
     h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
     r = requests.post(tgt,data=params,headers=h)
     st = r.text
     return st

In [2]:
import json
json.loads("test")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
    #################################################################################################
    # usage of the script
    # usage: python retrieve-cui-or-code.py -k APIKEY -v VERSION -i IDENTIFIER -s SOURCE
    # If you do not provide the -s parameter, the script assumes you are retrieving information for a
    # known UMLS CUI
    #################################################################################################
from Authentication import *
import requests
import json
import argparse
def retrive(identifier, username = 'ChaoZhang', password = '3f5hr7z9a0!!', version = '2019AA', source = 'ICD9CM'):

    ###################################
    #get TGT for our session
    ###################################
    AuthClient = Authentication(username, password)
    tgt = AuthClient.gettgt()
    uri = "https://uts-ws.nlm.nih.gov"

    try:
       source
    except NameError:
       source = None

    ##if we don't specify a source vocabulary, assume we're retrieving UMLS CUIs
    if source is None:
        content_endpoint = "/rest/content/"+str(version)+"/CUI/"+str(identifier)

    else:
        content_endpoint = "/rest/content/"+str(version)+"/source/"+str(source)+"/"+str(identifier)

    ##ticket is the only parameter needed for this call - paging does not come into play because we're only asking for one Json object
    query = {'ticket':AuthClient.getst(tgt)}
    r = requests.get(uri+content_endpoint,params=query)
    r.encoding = 'utf-8'
    items  = json.loads(r.text)
    jsonData = items["result"]

    ##uncomment the print statment if you want the raw json output, or you can just look at the documentation :=)
    #https://documentation.uts.nlm.nih.gov/rest/concept/index.html#sample-output
    #https://documentation.uts.nlm.nih.gov/rest/source-asserted-identifiers/index.html#sample-output
    #print (json.dumps(items, indent = 4))

    ############################
    ### Print out fields ####

    classType = jsonData["classType"]
    name = jsonData["name"]
    ui = jsonData["ui"]
    AtomCount = jsonData["atomCount"]
    Definitions = jsonData["definitions"]
    Atoms = jsonData["atoms"]
    DefaultPreferredAtom = jsonData["defaultPreferredAtom"]

    ## print out the shared data elements that are common to both the 'Concept' and 'SourceAtomCluster' class
    print ("classType: " + classType)
    print ("ui: " + ui)
    print ("Name: " + name)
    print ("AtomCount: " + str(AtomCount))
    print ("Atoms: " + Atoms)
    print ("Default Preferred Atom: " + DefaultPreferredAtom)

    ## These data elements may or may not exist depending on what class ('Concept' or 'SourceAtomCluster') you're dealing with so we check for each one.
    try:
       jsonData["definitions"]
       print ("definitions: " + jsonData["definitions"])
    except:
          pass

    try:
       jsonData["parents"]
       print ("parents: " + jsonData["parents"])
    except:
          pass

    try:
       jsonData["children"]
       print ("children: " + jsonData["children"])
    except:
          pass

    try:
       jsonData["relations"]
       print ("relations: " + jsonData["relations"])
    except:
          pass

    try:
       jsonData["descendants"]
       print ("descendants: " + jsonData["descendants"])
    except:
          pass

    try:
       jsonData["semanticTypes"]
       print("Semantic Types:")
       for stys in jsonData["semanticTypes"]:
           print("uri: "+ stys["uri"])
           print("name: "+ stys["name"])

    except:
          pass

ModuleNotFoundError: No module named 'Authentication'